In [23]:
!pip install --upgrade transformers datasets accelerate -q

import os
os.environ["WANDB_DISABLED"] = "true"

In [25]:
from google.colab import files
uploaded = files.upload()

Saving high_quality_passive_500.json to high_quality_passive_500.json


In [26]:
import json
from datasets import Dataset

with open("high_quality_passive_500.json") as f:
    raw_data = json.load(f)

dataset = Dataset.from_list(raw_data)
dataset = dataset.train_test_split(test_size=0.1)


In [29]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "gtrivedi/style-guide-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [30]:
def preprocess(example):
    inputs = tokenizer(example["prompt"], padding="max_length", truncation=True, max_length=512)
    targets = tokenizer(example["output"], padding="max_length", truncation=True, max_length=512)
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_dataset = dataset.map(preprocess, batched=True)

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [31]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./style-guide-fine-tuned",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=1,
    logging_dir='./logs',
    logging_steps=10,
    predict_with_generate=True
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [32]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
)

trainer.train()

<ipython-input-32-1b659f6a30c3>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss
10,0.025400
20,0.010700
30,0.006200
40,0.003300
50,0.003700
60,0.003300
70,0.001000
80,0.001000
90,0.001100
100,0.001000


TrainOutput(global_step=1125, training_loss=0.000592661527865049, metrics={'train_runtime': 218.5312, 'train_samples_per_second': 10.296, 'train_steps_per_second': 5.148, 'total_flos': 1540704043008000.0, 'train_loss': 0.000592661527865049, 'epoch': 5.0})

In [33]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Save model
trainer.save_model("/content/style-guide-base")
tokenizer.save_pretrained("/content/style-guide-base")


('/content/style-guide-base/tokenizer_config.json',
 '/content/style-guide-base/special_tokens_map.json',
 '/content/style-guide-base/spiece.model',
 '/content/style-guide-base/added_tokens.json',
 '/content/style-guide-base/tokenizer.json')

In [10]:
from huggingface_hub import notebook_login
notebook_login()


In [34]:
# Push back to your existing repo
model.push_to_hub("gtrivedi/style-guide-base",
                  commit_message="Fine-tuned on 500 multi-agent examples")
tokenizer.push_to_hub("gtrivedi/style-guide-base",
                      commit_message="Updated tokenizer")



model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/gtrivedi/style-guide-base/commit/eebe4d3577ef06374bedab4f6a43e741859748db', commit_message='Updated tokenizer', commit_description='', oid='eebe4d3577ef06374bedab4f6a43e741859748db', pr_url=None, repo_url=RepoUrl('https://huggingface.co/gtrivedi/style-guide-base', endpoint='https://huggingface.co', repo_type='model', repo_id='gtrivedi/style-guide-base'), pr_revision=None, pr_num=None)

In [12]:
from huggingface_hub import create_repo, upload_folder

create_repo("style-guide-base", repo_type="model", private=False)

upload_folder(
    repo_id="gtrivedi/style-guide-base",
    folder_path="./style-guide-base",
    commit_message="Initial upload of style writing assistant"
)


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/gtrivedi/style-guide-base/commit/9201209eddf060f613bed3b7c9fa7667e26bfa3b', commit_message='Initial upload of style writing assistant', commit_description='', oid='9201209eddf060f613bed3b7c9fa7667e26bfa3b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/gtrivedi/style-guide-base', endpoint='https://huggingface.co', repo_type='model', repo_id='gtrivedi/style-guide-base'), pr_revision=None, pr_num=None)